In [1]:
!pip install tf-keras


[notice] A new release of pip is available: 23.0.1 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [2]:
# ZenML
from zenml import pipeline, step

# General
import os
import numpy as np

# MongoDB
from pymongo import MongoClient

# Qdrant
from qdrant_client import QdrantClient
from qdrant_client.http.models import PointStruct, VectorParams, Distance

# Embedding Model
from sentence_transformers import SentenceTransformer

# For logging
import logging

logger = logging.getLogger(__name__)


In [3]:
@step
def load_data_from_mongodb() -> list:
    logger.info("Loading data from MongoDB...")
    
    client = MongoClient('mongodb://rag_mongodb:27017/')
    db = client['rag_db']
    collection = db['raw_data']
    
    documents = list(collection.find())
    logger.info(f"Loaded {len(documents)} documents from MongoDB.")
    
    return documents


In [4]:
@step
def categorize_and_preprocess_data(documents: list) -> list:
    logger.info("Categorizing and preprocessing data...")
    
    processed_data = []
    
    for doc in documents:
        content = doc['content']
        file_path = doc['path']
        source = doc['source']
        
        # Determine the category based on file extension
        if file_path.endswith('.md') or file_path.endswith('.rst'):
            category = 'article'
            # Additional preprocessing for articles if needed
        elif file_path.endswith('.py'):
            category = 'code'
            # Remove comments and docstrings if necessary
            content = remove_comments_and_docstrings(content)
        else:
            category = 'other'
        
        processed_data.append({
            'url': doc['url'],
            'path': file_path,
            'repository': doc['repository'],
            'branch': doc['branch'],
            'content': content,
            'source': source,
            'category': category
        })
    
    logger.info(f"Categorized and processed {len(processed_data)} documents.")
    return processed_data


In [5]:
import ast

def remove_comments_and_docstrings(source):
    """
    Remove comments and docstrings from Python source code.
    """
    try:
        parsed = ast.parse(source)
        for node in ast.walk(parsed):
            if isinstance(node, (ast.FunctionDef, ast.ClassDef, ast.AsyncFunctionDef, ast.Module)):
                if node.body and isinstance(node.body[0], ast.Expr):
                    if hasattr(node.body[0], 'value') and isinstance(node.body[0].value, ast.Str):
                        node.body = node.body[1:]
        return ast.unparse(parsed)
    except Exception as e:
        logger.warning(f"Error parsing Python code: {e}")
        return source  # Return original source if parsing fails


In [6]:
@step
def chunk_data(processed_data: list) -> list:
    logger.info("Chunking data...")
    
    chunked_data = []
    max_chunk_size = 500  # Adjust based on your embedding model's max input length
    
    for doc in processed_data:
        content = doc['content']
        doc_id = f"{doc['repository']}_{doc['path']}".replace('/', '_')
        
        # Split content into chunks
        content_length = len(content)
        chunks = [content[i:i+max_chunk_size] for i in range(0, content_length, max_chunk_size)]
        
        for idx, chunk in enumerate(chunks):
            chunked_data.append({
                'doc_id': str(doc_id),
                'chunk_id': f"{str(doc_id)}_{idx}",
                'chunk': chunk,
                'metadata': {
                    'url': doc['url'],
                    'path': doc['path'],
                    'repository': doc['repository'],
                    'branch': doc['branch'],
                    'source': doc['source'],
                    'category': doc['category']
                }
            })
    
    logger.info(f"Created {len(chunked_data)} chunks from documents.")
    return chunked_data


In [7]:
@step
def generate_embeddings(chunked_data: list) -> list:
    logger.info("Generating embeddings...")
    
    if not chunked_data:
        logger.warning("No data to generate embeddings for!")
        return []
    
    model = SentenceTransformer('all-MiniLM-L6-v2')
    batch_size = 32  # Adjust based on your hardware capabilities
    
    embeddings = []
    total_batches = (len(chunked_data) + batch_size - 1) // batch_size
    
    for i in range(0, len(chunked_data), batch_size):
        batch = chunked_data[i:i+batch_size]
        texts = [item['chunk'] for item in batch]
        try:
            batch_embeddings = model.encode(texts)
            for idx, item in enumerate(batch):
                item['embedding'] = batch_embeddings[idx].tolist()
            embeddings.extend(batch)
            logger.info(f"Processed batch {(i//batch_size)+1}/{total_batches}")
        except Exception as e:
            logger.error(f"Error generating embeddings for batch {i//batch_size}: {str(e)}")
            continue
    
    logger.info(f"Generated embeddings for {len(embeddings)} chunks.")
    return embeddings

In [8]:
@step
def store_embeddings_in_qdrant(chunked_data: list):
    logger.info("Storing embeddings in Qdrant...")
    
    try:
        client = QdrantClient(host='rag_qdrant', port=6333)
        
        # Define collection parameters
        collection_name = 'rag_collection'
        vector_size = len(chunked_data[0]['embedding'])
        
        # Check if collection exists and recreate
        try:
            client.get_collection(collection_name)
            client.delete_collection(collection_name)
            logger.info(f"Deleted existing collection: {collection_name}")
        except Exception:
            pass
        
        client.create_collection(
            collection_name=collection_name,
            vectors_config=VectorParams(size=vector_size, distance=Distance.COSINE)
        )
        
        # Prepare data for Qdrant
        batch_size = 100  # Adjust based on your needs
        for i in range(0, len(chunked_data), batch_size):
            batch = chunked_data[i:i+batch_size]
            points = []
            for idx, item in enumerate(batch):
                # Generate a positive integer ID using the position in the dataset
                point_id = i * batch_size + idx + 1  # Ensures positive, unique IDs starting from 1
                
                point = PointStruct(
                    id=point_id,  # Use the positive integer ID
                    vector=item['embedding'],
                    payload={
                        **item['metadata'],
                        'chunk_id': item['chunk_id'],
                        'doc_id': item['doc_id'],
                        'chunk': item['chunk']
                    }
                )
                points.append(point)
            
            try:
                # Upload batch to Qdrant
                client.upsert(
                    collection_name=collection_name,
                    points=points
                )
                logger.info(f"Uploaded batch of {len(points)} embeddings to Qdrant (IDs {points[0].id} to {points[-1].id})")
            except Exception as e:
                logger.error(f"Error uploading batch: {str(e)}")
                # Log the first failing point for debugging
                if points:
                    logger.error(f"First point in failing batch - ID: {points[0].id}")
                raise
        
        logger.info(f"Successfully stored all {len(chunked_data)} embeddings in Qdrant.")
    except Exception as e:
        logger.error(f"Error storing embeddings in Qdrant: {str(e)}")
        raise

In [13]:
# First, create a global variable to store the results
pipeline_results = None

@pipeline
def featurization_pipeline():
    try:
        documents = load_data_from_mongodb()
        if not documents:
            logger.error("No documents loaded from MongoDB!")
            return None
            
        processed_data = categorize_and_preprocess_data(documents)
        if not processed_data:
            logger.error("No documents after preprocessing!")
            return None
            
        chunked_data = chunk_data(processed_data)
        if not chunked_data:
            logger.error("No chunks created!")
            return None
            
        chunked_data_with_embeddings = generate_embeddings(chunked_data)
        if not chunked_data_with_embeddings:
            logger.error("No embeddings generated!")
            return None
            
        store_embeddings_in_qdrant(chunked_data_with_embeddings)
        logger.info("Pipeline completed successfully!")
        
        # Return the results
        return chunked_data_with_embeddings
    except Exception as e:
        logger.error(f"Pipeline failed: {str(e)}")
        raise

# Run the pipeline and store results
pipeline_instance = featurization_pipeline()



Pipeline completed successfully!
Initiating a new run for the pipeline: featurization_pipeline.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml login --local.
Using cached version of step load_data_from_mongodb.
Using cached version of step categorize_and_preprocess_data.
Using cached version of step chunk_data.
Using cached version of step generate_embeddings.
Step store_embeddings_in_qdrant has started.
Storing embeddings in Qdrant...
HTTP Request: GET http://rag_qdrant:6333/collections/rag_collection "HTTP/1.1 200 OK"
HTTP Request: DELETE http://rag_qdrant:6333/collections/rag_collection "HTTP/1.1 200 OK"
Deleted existing collection: rag_collection
HTTP Request: PUT http://rag_qdrant:6333/collections/rag_collection "HTTP/1.1 200 OK"
HTTP Request: PUT http://rag_qdrant:6333/collections/rag_collection/points?wait=true "HTTP/1.1 200 OK"
Up

In [14]:
# Now test the results
def search_qdrant(query_text: str, limit: int = 3):
    try:
        # Connect to Qdrant
        qdrant_client = QdrantClient(host='rag_qdrant', port=6333)
        
        # Load the same model used in the pipeline
        model = SentenceTransformer('all-MiniLM-L6-v2')
        
        # Generate embedding for the query
        query_vector = model.encode(query_text)
        
        # Ensure the query vector is a list of floats
        if not isinstance(query_vector, list):
            query_vector = query_vector.tolist()
        
        # Log the query vector for debugging
        logger.debug(f"Query vector: {query_vector[:10]}...")  # Log first 10 elements
        
        # Search
        search_results = qdrant_client.search(
            collection_name='rag_collection',
            query_vector=query_vector,
            limit=limit
        )
        
        # Display results
        print(f"\nSearch Results for: '{query_text}'")
        print("-" * 50)
        for result in search_results:
            print(f"Score: {result.score:.4f}")
            print(f"Repository: {result.payload.get('repository')}")
            print(f"Path: {result.payload.get('path')}")
            print(f"Category: {result.payload.get('category')}")
            print(f"Chunk: {result.payload.get('chunk')[:200]}...")  # Show first 200 chars
            print("-" * 50)
            
    except Exception as e:
        logger.error(f"Error searching Qdrant: {str(e)}")
        raise

# Example usage:
search_qdrant("What is ROS?")

Use pytorch device_name: cpu
Load pretrained SentenceTransformer: all-MiniLM-L6-v2


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

HTTP Request: POST http://rag_qdrant:6333/collections/rag_collection/points/search "HTTP/1.1 200 OK"

Search Results for: 'What is ROS?'
--------------------------------------------------
Score: 0.6981
Repository: ros2/ros2_documentation
Path: source/Releases/Alpha-Overview.rst
Category: article
Chunk: ----------------------------------------------------- .. contents:: Table of Contents :local: Background ^^^^^^^^^^ As explained in a `design article <https://design.ros2.org/articles/why_ros2.html>`_...
--------------------------------------------------
Score: 0.6934
Repository: ros2/ros2_documentation
Path: source/Releases/Alpha-Overview.rst
Category: article
Chunk: luding the core libraries and core command line tools. Pretty much anything not listed above is not included in this release. The next steps are described in the `Roadmap <../../The-ROS2-Project/Roadm...
--------------------------------------------------
Score: 0.6873
Repository: ros2/examples
Path: README.md
Category: arti